<a href="https://colab.research.google.com/github/jouvetg/igm/blob/main/notebooks/IGM_aletsch_simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### <h1 align="center" id="title">IGM notebook tutorial (aletsch-simple) </h1>

This tutorial aims to guide you modelling the great Aletsch Glacier, Switzerland, in a very simple set-up with the Instructed Glacier Model (IGM).

Let us firt download the IGM code, ice flow emulator, and input data

In [ ]:
# get the core source of igm (in igm.py)
!wget -nv -O igm.py https://raw.githubusercontent.com/jouvetg/igm/main/src/igm.py

# get input data
!wget -nv -O geology.nc https://raw.githubusercontent.com/jouvetg/igm/main/examples/aletsch-simple/geology.nc
!wget -nv -O mb_simple_param.txt https://raw.githubusercontent.com/jouvetg/igm/main/examples/aletsch-simple/mb_simple_param.txt

# get the ice flow emulator
!apt install subversion
!svn export https://github.com/jouvetg/igm/trunk/model-lib/f15_cfsflow_GJ_22_a


In the first set-up, we take top ice surface topography and (zero) ice thickness defined in geology.nc, and force IGM with mass balance parameters defined in mb_simple_param.txt.

First, we import necessary libraries, check the version of tensorflow, and inport the class igm (defined in igm.py), and create an igm object.

In [ ]:
import sys
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
print('TF version is ',tf.__version__)
sys.argv = ['']  # this is absolutly necessary in Jupyter notebook
from igm import Igm
glacier=Igm() # create an object glacier

Altough, you don't have to, it is highly recommended to run under GPU (click above 'Runtime' -> 'Change Runtime type' -> Activate GPU). Then the fowllowing comand check that a GPU is indeed available, and give the name of the GPU. (With the free veryion of colab these GPU are often old and not that efficient -- like the K80 --, IGM would typically run faster on new material)

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

Next, we overide the default configuration parameters, initialize the class, and check at the parameters

In [ ]:
glacier.config.tstart            = 1000  # we start at time t=1000
glacier.config.tend              = 1250  # we end at time   t=1250
glacier.config.tsave             = 5     # we save result each 5 years
glacier.config.init_slidingco    = 12
glacier.config.init_arrhenius    = 78
glacier.config.iceflow_model_lib_path  = 'f15_cfsflow_GJ_22_a' # we provide the path for th ice flow emultor
glacier.config.type_mass_balance = 'simple' # we use the 'simple' ELA-parametrized SMB model
glacier.config.mb_simple_file    = 'mb_simple_param.txt' # this is the file where are defined the SMB params.
glacier.config.usegpu            = True # we do not use the GPU for the first run
glacier.config.plot_live         = False # we do not plot the result life, but vizualize a posteriori
glacier.config.plot_result       = False # we do not plot the result life, but vizualize a posteriori

Now we run the IGM time evolution workflow :


In [ ]:
glacier.initialize() 
with tf.device(glacier.device_name):
    glacier.load_ncdf_data(glacier.config.geology_file)
    glacier.initialize_fields()               
    while glacier.t < glacier.config.tend:                       
        glacier.update_smb()
        glacier.update_iceflow()
        glacier.update_t_dt() 
        glacier.update_thk()       
        glacier.update_ncdf_ex()
        glacier.update_ncdf_ts()
        glacier.update_plot()
        glacier.print_info()
        
glacier.print_all_comp_info()

We have now modelled 250 years of evolution of the Great Aletsch Glacier in less than a minute! (this would take < 15 sec on modern GPU). IGM monitors key variables (ice volume, time step) during computation you can monitor, and gives a statatistic of the computational cost of each mdel components at the end. The results were stored in ex.nc. Now we can vizualize the results (ice thickness, ice speed, or more) in form of an animation.



In [ ]:
igm.animate_result('ex.nc','thk',save=True)
#igm.animate_result('ex.nc','velsurf_mag',save=True)

igm.config.type_mass_balance = 'sinus'
Now, we may custumize your our mass balance function with an oscillating ELA. For that, you can define a function of the class igm, and make sure to activate the corresponding keywork in igm.config.type_mass_balance

In [ ]:
!rm ex.nc

In [ ]:
from igm import Igm
import math

class Igm(Igm):

    def update_smb_sinus(self):
        """
            mass balance 'sinus'
        """

        ela     = 2800 + 500*math.sin((self.t/50)*math.pi)
        gradabl = 0.009
        gradacc = 0.005
        maxacc  = 2.0

        smb = self.usurf - ela
        smb *= tf.where(tf.less(smb, 0), gradabl, gradacc)
        smb = tf.clip_by_value(smb, -100, maxacc)
        smb = tf.where(self.icemask>0.5, smb, -10 )

        self.smb.assign( smb )

glacier = Igm()

glacier.config.tstart                 = 0
glacier.config.tend                   = 500
glacier.config.tsave                  = 2
glacier.config.cfl                    = 0.5
glacier.config.init_slidingco         = 12
glacier.config.init_arrhenius         = 78
glacier.config.iceflow_model_lib_path = 'f15_cfsflow_GJ_22_a'
glacier.config.type_mass_balance      = 'sinus'
glacier.config.usegpu                 = True

glacier.initialize() 
with tf.device(glacier.device_name):
    glacier.load_ncdf_data(igm.config.geology_file)
    glacier.initialize_fields()               
    while glacier.t < glacier.config.tend:                       
        glacier.update_smb()
        glacier.update_iceflow()
        glacier.update_t_dt() 
        glacier.update_thk()       
        glacier.update_ncdf_ex()
        glacier.update_ncdf_ts()
        glacier.update_plot()
        glacier.print_info()
        
glacier.print_all_comp_info()